# Guided Project: Popular Data Science Questions 

## Table of Contents 
1. [Introduction](#introduction)
2. [Stack Exchange](#stack-exchange)
3. [Stack Exchange Data Explorer](#sede)
4. [Getting the Data](#getting-data)
5. [Import the dependencies](#import-the-dependencies)
6. [Exploring the Data](#exploring-data)
7. [Cleaning the Data](#cleaning-data)
8. [Most Used and Most Viewed](#most)
9. [Relations Between Tags](#relations-tags)
10. [Enter Domain Knowledge](#enter-domain)
11. [Just a Fad?](#just-fad)
12. [Next Steps](#next-steps)







## Introduction <a name = "introduction"></a>

This is a guided project I completed on the Dataquest learning platform. The focus of this project was to learn new skills related to t.... The goals are to ... the available public data set based on .... The code is written in Python, using ... libraries, and is shown in Jupyter Notebooks.

To run these notebooks in your own local dev environent:

1. Download the ....ipynb file in this repository and place it in  "<your_dev_folder>" folder
2. Download the data set ....csv file in this repository and place it in "<your_dev_folder>/data/..." folder

For the full data set and story behind it, published by ..., go to the following source:

### Learn
In this course, we explored the business context in which data science happens. You'll now have a chance to apply what you've learned in a more relaxed way. Feel free to explore alternatives to the path we're taking in this project.

In this scenario, you're working for a company that creates data science content, be it books, online articles, videos or interactive text-based platforms like Dataquest.

You're tasked with figuring out what is best content to write about. Because you took this course, you know that given the lack of instructions there's some leeway in what "best" means here.

Since you're passionate about helping people learn, you decide to scour the internet in search for the answer to the question "What is it that people want to learn about in data science?" (as opposed to determining the most profitable content, for instance).

Thinking back to your experience when you first started learning programming, it occurs to you that if you wanted to figure out what programming content to write, you could consult Stack Overflow (a question and answer website about programming) and see what kind of content is more popular.

You decide to investigate Stack Overflow a little more and find out that it is part of a question and answer website network called Stack Exchange.


## Stack Exchange <a name = "stack-exchange"></a>

### Learn

Stack Exchange hosts sites on a multitude of fields and subjects, including mathematics, physics, philosophy, and data science! Here's a sample of the most popular sites:


Stack Exchange employs a reputation award system for its questions and answers. Each post — each question/answer — is a post that is subject to upvotes and downvotes. This ensures that good posts are easily identifiable.

If you're not familiar with Stack Overflow or any other Stack Exchange site, you can check out this tour.

Being a multidisciplinary field, there a few Stack Exchange websites there are relevant to our goal here:

    Data Science
    Cross Validated — a statistics site
    Artificial Intelligence
    Mathematics
    Stack Overflow

And if we want to include Data Engineering, we can also consider:

    Database Administrators;
    Unix & Linux;
    Software Engineering;

If you open the link in the image shared above, you'll find a complete list of Stack Exchange websites sorted by percentage of questions that received answers. At the time of this writing, Data Science Stack Exchange (DSSE) is on the bottom 10 sites with respect to this metric.

The fact that DSSE is a data science dedicated site (contrarily to the others), coupled with it having a lot of unanswered questions, makes it an ideal candidate for this investigation. DSSE will be the focus of this guided project.

In this screen you'll be doing your first off-Dataquest exercise!

### Instructions

1. If you're not familiar with any Stack Exchange website, take the time to explore one of them. Try to answer a few of these questions in a markdown cell:
- What kind of questions are welcome on this site?
- What, other than questions, does the site's home subdivide into? Does any of them look useful towards our goal?
- What information is available in each post?
2. Explore some of the questions that were asked.
3. Try asking a couple of questions on any of Stack Exchange sites to get a better feel for how the sites operate.


## Stack Exchange Data Explorer <a name = "sede"></a>

### Learn

After a spending some time investigating the website, you decide that the tags will be very useful in categorizing content, saving you the trouble of you having to do it yourself.

Now comes the challenge of accessing the data en masse. One potential solution would be to scrape the site. However, because we still haven't learned how to web scrape, and because we have an easier alternative (mostly the second reason), we're going to do something else.

Stack Exchange provides a public data base for each of its websites. [Here's](https://data.stackexchange.com/datascience/query/new) a link to query and explore Data Science Stack Exchange's database.

You can read more about Stack Exchange Data Explorer (SEDE) on its [help section](https://data.stackexchange.com/help) and on [this](https://data.stackexchange.com/tutorial) tutorial link.


Note that SEDE uses a different dialect (Transact-SQL — Microsoft's SQL) than SQLite , which you learned earlier. Most things are the same, but some are different. For instance, the query below selects the top 10 results from a query.

SELECT TOP 10 *

  FROM tags

 ORDER BY Count DESC;

In SQLite we would not only use the keyword LIMIT instead of TOP we would also included it at the end of the query, instead of in the SELECT statement. If you run into any issues due to these differences, try to research on your own how to solve them. Here's a helpful resource.

### Instructions

1. Explore Data Science Stack Exchange's data model.
- Investigate a few of the tables, especially those whose names sound more promising;
- Write a few queries to get a feel for the data;
2. In a markdown cell, write about what tables look more promising towards finding the most popular content.


## Getting the Data <a name = "getting-data"></a>

### Learn

The posts table has a lot of columns. We'll be focusing our attention on those that seem relevant towards our goal:

    Id: An identification number for the post.

    PostTypeId: An identification number for the type of post.

    posttypes

    CreationDate: The date and time of creation of the post.
    Score: The post's score.
    ViewCount: How many times the post was viewed.
    Tags: What tags were used.
    AnswerCount: How many answers the question got (only applicable to question posts).
    FavoriteCount: How many times the question was favored (only applicable to question posts).

Note that with the exception of the tags column, the last few columns contain information about how popular the post is — the kind of information we're after.

There are eight different types of post. Before we try to figure out which of them are relevant to us, let's check how many of them there are:

SELECT PostTypeId, COUNT(*) as NrOfPosts

  FROM posts

 GROUP BY PostTypeId;

PostTypeId 	NrOfPosts
1 	21446
2 	23673
4 	236
5 	236
6 	11
7 	1

Due to their low volume, anything that isn't questions or answers is mostly inconsequential. Even if it happens to be the case that such kind of posts is immensely popular, they would just be outliers and not relevant to us. We'll then just focus on the questions.

Since we're only interested in recent posts, we'll limit our analysis to the posts of 2019. (At the time of writing it is early 2020).

The dataset we'll be using in this guided project is one resulting from a possible solution to the following exercise.

### Instructions

1. Run a query against the SEDE DSSE database that extracts the columns listed above for all the questions in 2019.


## Import the dependencies <a name="import-the-dependencies"></a>
We need to use the ... libraries within this notebook.

## Exploring the Data <a name = "exploring-data"></a>

## Learn

The result of the query was stored in a file called 2019_questions.csv. Here are the first few rows of the data we got:

Id,PostTypeId,CreationDate,Score,ViewCount,Tags,AnswerCount,FavoriteCount

"44419","1","2019-01-23 09:21:13","1","21","<machine-learning><data-mining>","0",""

"44420","1","2019-01-23 09:34:01","0","25","<machine-learning><regression><linear-regression><regularization>","0",""

"44423","1","2019-01-23 09:58:41","2","1651","<python><time-series><forecast><forecasting>","0",""

"44427","1","2019-01-23 10:57:09","0","55","<machine-learning><scikit-learn><pca>","1",""

"44428","1","2019-01-23 11:02:15","0","19","<dataset><bigdata><data><speech-to-text>","0",""

Looking at the of each row, it stands out that FavouriteCount has missing values. What other issues are there with the data? Let's explore it.

## Instructions

1. Read in the file into a dataframe.
2. Explore the data. Try to answer a few of these questions in a markdown cell:
- How many missing values are there in each column?
- Can we fix the missing values somehow?
- Are the types of each column adequate?
- What can we do about the Tags column?


## Cleaning the Data <a name = "cleaning-data"></a>

### Learn

On the previous screen, we identified issues with the data. Fortunately for us, the folks at Stack Exchange did a great job of providing clean data. Let's fix the one issue we found, set the appropriate types for the columns, and clean the Tags column to fit our purposes.

At the end of this screen, the types of the columns should be as follows.

Id                        int64

CreationDate     datetime64[ns]

Score                     int64

ViewCount                 int64

Tags                     object

AnswerCount               int64

FavoriteCount             int64

The values in the Tags column are strings that look like this:

"<machine-learning><regression><linear-regression><regularization>"

We'll want to transform this string in something more suitable to use typical string methods. Our goal will be to transform strings like the above in something like:

"machine-learning,regression,linear-regression,regularization"

We can then split on , to obtain a list.

### Instructions

1. Fill in the missing values with 0.
2. Set the types of each column in accordance to what was illustrated above.
3. Clean the Tags column and assign it back to itself:
- Use the process illustrated above.
- Assign the result to questions["Tags"].


## Most Used and Most Viewed <a name = "most"></a>

### Learn

We now focus on determining the most popular tags. We'll do so by considering two different popularity proxies: for each tag we'll count how many times the tag was used, and how many times a question with that tag was viewed.

We could take into account the score, or whether or not a question is part of someone's favorite questions. These are all reasonable options to investigate; but we'll limit the focus of our research to counts and views for now.

### Instructions

1. Count how many times each tag was used.
2. Count how many times each tag was viewed.
3. Create visualizations for the top tags of each of the above results.


## Relations Between Tags <a name = "relations-tags"></a>

### Learn

On this screen

    most_used is a dataframe that counts how many times each of the top 20 tags was used.
    most_viewed is a dataframe that counts how many times each of the top 20 tags was viewed.

Looking at the results from the last exercise, we see that most top tags are present in both dataframes.

Let's see what tags are in most_used, but not in most_viewed. We can identify them by the missing values in ViewCount below.

in_used = pd.merge(most_used, most_viewed, how="left", left_index=True, right_index=True)

	Count 	ViewCount
machine-learning-model 	224 	NaN
statistics 	234 	NaN
clustering 	257 	33928.0
predictive-modeling 	265 	NaN
r 	268 	NaN
dataset 	340 	43151.0
regression 	347 	49451.0
pandas 	354 	201787.0
lstm 	402 	74458.0
time-series 	466 	64134.0
cnn 	489 	70349.0
nlp 	493 	71382.0
scikit-learn 	540 	128110.0
tensorflow 	584 	121369.0
classification 	685 	104457.0
keras 	935 	268608.0
neural-network 	1055 	185367.0
deep-learning 	1220 	233628.0
python 	1814 	537585.0
machine-learning 	2693 	388499.0

Similarly, let's see what tags are in the latter, but not the former:

pd.merge(most_used, most_viewed, how="right", left_index=True, right_index=True)

	Count 	ViewCount
clustering 	257.0 	33928
csv 	NaN 	38654
pytorch 	NaN 	40240
dataset 	340.0 	43151
regression 	347.0 	49451
numpy 	NaN 	49767
time-series 	466.0 	64134
cnn 	489.0 	70349
nlp 	493.0 	71382
lstm 	402.0 	74458
dataframe 	NaN 	89352
classification 	685.0 	104457
tensorflow 	584.0 	121369
scikit-learn 	540.0 	128110
neural-network 	1055.0 	185367
pandas 	354.0 	201787
deep-learning 	1220.0 	233628
keras 	935.0 	268608
machine-learning 	2693.0 	388499
python 	1814.0 	537585

The tags present in most_used and not present in most_viewed are:

    machine-learning-model
    statistics
    predictive-modeling
    r

And the tags present in most_viewed but not in most_used are:

    csv
    pytorch
    dataframe

Some tags also stand out as being related. For example, python is related to pandas, as we can find both pythons and pandas in the same country — or better yet, because pandas is a Python library. So by writing about pandas, we can actually simultaneously tackle two tags.

Other pairs of tags, shouldn't be related at all, like pandas and r:

questions[questions["Tags"].apply(

    lambda tags: True if "r" in tags and "pandas" in tags else False)

]

	Id 	CreationDate 	Score 	ViewCount 	Tags 	AnswerCount 	FavoriteCount
2873 	60074 	2019-09-11 20:35:17 	0 	22 	[r, pandas, dplyr] 	0 	0
3651 	49148 	2019-04-11 19:41:39 	1 	83 	[r, data-mining, pandas, matlab, databases] 	3 	0

Just two results. You can look at these questions by replacing ID in https://datascience.stackexchange.com/questions/ID with the questions' Ids values and see what they are about.

### Instructions

The goal of this exercise is to make you think about technical solutions to determining how tags are related. We haven't covered any techniques to deal with this yet, so don't worry if nothing comes to mind.

1. Brainstorm some ways in which you could find relationships between pair of tags.
2. Brainstorm some ways in which you could find relationships between multiple tags.


## Enter Domain Knowledge <a name = "enter-domain"></a>

### Learn

Although analytical solutions to the problem set forth in the previous screen exist, they aren't necessarily needed, nor the best way to go about solving the problem.

Domain knowledge can be very useful. Let's see how it can help us here.

You may have noticed that the most used tags are also the most viewed. From the top 10 tags of each, here's a list of the tags in common: python, machine-learning, deep-learning, neural-network, keras, tensorflow, classification, scikit-learn.

Do you know what each of these or most of these tags means? Could there be strong relations between them?

### Instructions

1. Take a look at the tags page on DSSE. Try to answer the questions above. Do additional research on the internet as needed.

## Just a Fad? <a name = "just-fad"></a>

### Learn

Before we officially make our recommendation, it would be nice to solidify our findings with additional proof. More specifically, one thing that comes to mind is "Is deep learning just a fad?" Ideally, the content we decide to create will be the most useful for as long as possible. Could interest in deep learning be slowing down? Back to SEDE!

The file all_questions.csv holds the result of the query below — this query fetches all of the questions ever asked on DSSE, their dates and tags.

SELECT Id, CreationDate, Tags

  FROM posts

 WHERE PostTypeId = 1;

In this we will track the interest in deep learning across time. We will:

    Count how many deep learning questions are asked per time period.
    The total amount of questions per time period.
    How many deep learning questions there are relative to the total amount of questions per time period.

### Instructions

1. Read the file all_questions.csv into a dataframe.
2. Transform the tags column in a similar manner to what was previously done.
3. Think about what questions should be classified as deep learning questions and the implement that definition.
4. Decide on an adequate timeframe and track interest in deep learning across that timeframe:
- Count how many deep learning questions are asked per time period.
- The total amount of questions per time period.
- How many deep learning questions there are relative to the total amount of questions per time period;
- Write your observations and final recommendation in a markdown cell.

## Next Steps <a name = "next-steps"></a>

### Learn

Our work allowed us to conclude that deep learning is the most popular topic right now. Here are some things to consider:

- What other content can we recommend that isn't as popular? You can try using association rules to find strong relations between tags.
- What other popularity features could we include in our analysis? Perhaps scores and favourite counts?
- We focused on DSSE. How could we use other related sites to help us with our goal?
- How can we leverage other sites to determine what non-data-science content to write about? For example, is there some mathematical field that leads to more questions than others?